In [7]:
import pandas as pd
import numpy as np
from sqlalchemy import MetaData, create_engine, Table, Column, ForeignKey
from sqlalchemy import Integer, String, Date, Float, Time
from sqlalchemy import exc
from prettytable import PrettyTable

In [8]:
#Constantes
DB = "tp312"
HOST = "localhost"
PORT = "5433"
PASS = "postgres"
USER = "postgres"

# Diagrama de DW  - Inmobiliaria

![Diagrama](img/Inmobiliaria.png "Diagrama")

In [9]:
url_conect = USER + ':' + PASS + '@' + HOST+ ':' + PORT + '/'+ DB
engine = create_engine('postgresql://' + url_conect)

In [11]:
metadata = MetaData()


estate = Table('estate', metadata,
              Column('id', Integer, primary_key=True),
              Column('category', String),
              Column('area', Float, nullable=False),
              Column('city', String, nullable=False),
              Column('province', String, nullable=False),
              Column('rooms', Integer, nullable=False),
              Column('bedrooms', Integer, nullable=False),
              Column('garage', Integer),
              Column('meters', Float, nullable=False))

customer = Table('customer', metadata,
                Column('id', Integer, primary_key=True),
                Column('name', String, nullable=False),
                Column('surname', String, nullable=False),
                Column('budget', Float),
                Column('address', String),
                Column('city', String),
                Column('phone', String))

visit = Table('visit', metadata, 
             Column('id', Integer, primary_key=True),
             Column('duration', Integer, nullable=False))

sale = Table('sale', metadata, 
            Column('id', Integer, primary_key=True),
            Column('agreed_price', Float, nullable=False),
            Column('status', String, nullable=False))

fact = Table('fact', metadata,
            Column('id', Integer, primary_key=True),
            Column('id_estate', Integer, ForeignKey('estate.id')),
            Column('id_cust', Integer, ForeignKey('customer.id')),
            Column('date', Date, nullable=False),
            Column('id_visit', Integer, ForeignKey('visit.id'), nullable=True),
            Column('id_sale', Integer, ForeignKey('sale.id'), nullable=True))


In [12]:
metadata.create_all(engine)

Carga de Datos
--------------

In [13]:
try:
    df_estate = pd.read_csv('dataTp3/Inmobilaria/Estate.csv')
    df_estate.to_sql('estate', engine, index=None, if_exists="append")
    
except exc.IntegrityError:
    print("Los datos de Estate ya han sido cargados en la BD")

In [14]:
try:
    df_customer = pd.read_csv('dataTp3/Inmobilaria/Customer.csv')
    df_customer.to_sql('customer', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Customer ya han sido cargados en la BD")  

In [15]:
try:
    df_visit = pd.read_csv('dataTp3/Inmobilaria/Visit.csv')
    df_visit.to_sql('visit', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Visit ya han sido cargados en la BD")

In [16]:
try:
    df_sale = pd.read_csv('dataTp3/Inmobilaria/Sale.csv')
    df_sale.to_sql('sale', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Sale ya han sido cargados en la BD")

In [17]:
try:
    df_fact = pd.read_csv('dataTp3/Inmobilaria/Fact.csv')
    df_fact.to_sql('fact', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Fact ya han sido cargados en la BD")

# Consultas

## Consulta 1: 
¿Qué tipo de propiedad se vendió por el precio más alto con respecto a cada
ciudad y meses?

In [67]:
query = 'select date as mes, city, category, agreed_price from fact\
        inner join estate on (id_estate = estate.id)\
        inner join sale on (id_sale = sale.id)\
        where id_sale is not NULL'

resul = pd.read_sql_query(query, con=engine)

resul["mes"] = pd.DatetimeIndex(resul["mes"]).month

cities = resul.city.drop_duplicates().values

x = PrettyTable()
x.field_names = ["City", "Mes", "Categoty", "Price"]
for city in cities:
    data_city = resul[resul["city"] == city]
    meses = data_city.mes.drop_duplicates().values
    for mes in meses:
        data_mes = data_city[data_city["mes"] == mes]
        data_mes.sort_values(by=["agreed_price"], ascending=False)
        precio = data_mes.agreed_price.values[0]
        catg = data_mes.category.values[0]
        x.add_row([city, mes, catg, precio])
print(x)

+----------------------------------------+-----+--------------+-----------+
|                  City                  | Mes |   Categoty   |   Price   |
+----------------------------------------+-----+--------------+-----------+
|               Schaumburg               |  10 | departamento | 2854306.0 |
|                 Calape                 |  12 | departamento | 9282258.0 |
|               Bobrowice                |  8  | departamento | 4335467.0 |
|                Shuguang                |  12 | monoambiente | 4353605.0 |
|          Jabłonowo Pomorskie           |  3  |     casa     | 9218195.0 |
|                  Aku                   |  1  | departamento | 4635670.0 |
|                 Suchań                 |  8  |    duplex    | 9381036.0 |
|                Xingfeng                |  5  | departamento | 2741920.0 |
|                 Baratã                 |  2  | monoambiente | 6637962.0 |
|                Donghai                 |  8  |   triplex    | 7318446.0 |
|           

## Consulta 1 - V2:

¿Qué tipo de propiedad se vendió por el precio más alto con respecto a cada
ciudad y meses?


In [19]:
string_sql = """select city, date_part('month',"date") as mes, category, MAX(agreed_price) as Price_Max from fact
                inner join estate on (id_estate = estate.id)
                inner join sale on (id_sale = sale.id)
                where id_sale is not NULL
                group by city, category, mes
                order by city ASC; """

df_propiedades = pd.read_sql_query(string_sql,engine.connect()) 
df_propiedades

,city,mes,category,price_max
0,Abomey,12.0,departamento,4981076.0
1,Ai Tu,5.0,monoambiente,7053829.0
2,Ajung,9.0,departamento,1354858.0
3,Akron,9.0,departamento,4055170.0
4,Aku,1.0,departamento,4635670.0
5,Akunk’,11.0,duplex,8815758.0
6,Alaghsas,3.0,departamento,1979113.0
7,Alameda,7.0,departamento,2605728.0
8,Álftanes,4.0,departamento,9230622.0
9,Al Ḩarāk,7.0,monoambiente,4840501.0


# Consulta 2:
¿Quén ha comprado un piso con el precio más alto con respecto a cada mes?

In [55]:
string_sql = """select mes, name, surname, agreed_price from (
                    select mes, name, surname, agreed_price, rank() over particion_por_mes
                        from ( select date_part('month',"date") as mes, "name", surname, category, agreed_price
                            from fact inner join estate on (id_estate = estate.id)
                            inner join sale on (id_sale = sale.id) 
                            inner join customer on (id_cust = customer.id)
                        where id_sale is not NULL and category = 'piso'
                        order by agreed_price DESC) as pepe
                        WINDOW particion_por_mes AS (partition by mes order by agreed_price DESC)
                ) as sub_query
                WHERE rank = 1; """

df_clientes_pisos = pd.read_sql_query(string_sql,engine.connect()) 
df_clientes_pisos


,mes,name,surname,agreed_price
0,1,Samuel,Cook,8116995
1,2,Debra,Dunn,6368893
2,3,Julia,Mitchell,9766483
3,4,Paul,Price,5946760
4,5,Victor,Elliott,9911554
5,6,Marie,Thomas,5854180
6,7,Pamela,Payne,6114295
7,8,Richard,Rose,6010489
8,10,Janice,George,7998435
9,11,Carl,Murray,4502727


## Consulta 3:
¿Cuál es la duración media de visitas en las propiedades de cada categoría?

In [57]:
string_sql = """ select distinct category, tiempo_promedio
                 from (
                        select category, avg(duration)
                        over particion_por_categoria as tiempo_promedio
                        from (
                                fact inner join estate on (id_estate = estate.id) 
                                inner join visit on (id_visit = visit.id) )
                        WINDOW particion_por_categoria AS (partition by category)
                 ) as promedio; """
df_mayor_duracion = pd.read_sql_query(string_sql,engine.connect()) 
df_mayor_duracion

##Tiempo promedio en horas

,category,tiempo_promedio
0,departamento,5.319231
1,casa,4.750000
2,duplex,6.375000
3,monoambiente,4.917355
4,triplex,5.250000
5,piso,5.000000
